In [1]:
import duckdb
from tqdm.auto import tqdm 

connection_source = duckdb.connect(database="../dbt/database_name.duckdb", read_only = True)
connection_destination = duckdb.connect(database="../dbt/metabase.duckdb")

In [2]:
# Schéma source et destination
source_schema = "main_marts"
destination_schema = "main"


In [3]:
common_tables = ["marts_common_cities"]

production_tables = ["marts_stats_monthly_global",
          "marts_production_housing",
          "marts_production_establishments",
          "marts_production_owners",
          "marts_production_users",
          "marts_production_campaigns"    
]

join_tables = [
    "marts_production_join_campaigns_housing",
    "marts_production_join_establishment_cities",
    "marts_production_join_housing_groups",
    "marts_production_join_owner_housing",
]

ff_tables = [
    "marts_ff_ext_2020_geo_code",
    "marts_ff_ext_2023_geo_code",
]

lovac_tables = [
     "marts_lovac_2020_geo_code",
     "marts_lovac_2023_geo_code",
     "marts_lovac_2024_geo_code",
     "marts_lovac_exh_2020",
     "marts_lovac_exh_2023",
     "marts_lovac_exh_2024",
] 


mapping = {
    "marts_production_join_campaigns_housing" : "join_campaigns_housing",
    "marts_production_join_establishment_cities": "join_establishment_cities",
    "marts_production_join_housing_groups": "join_housing_groups",
    "marts_production_join_owner_housing": "join_owner_housing"
}
# tables = ["marts_common_cities"]

In [4]:
filters = []

In [5]:
tables = production_tables + join_tables + ff_tables + lovac_tables + common_tables

if filters:
    tables = filters

In [6]:
import numpy as np

In [7]:

# Boucle sur chaque table pour transférer les données
for table in tqdm(tables, desc="Transfert des tables"):
    # Lecture des données de la table source
    query_read = f'SELECT * FROM {source_schema}.{table}'
    df = connection_source.execute(query_read).fetchdf()
    df = df.replace({None: np.nan})

    # Ecriture des données dans la table de destination
    destionation_table = mapping.get(table, table)
    query_write = f'CREATE OR REPLACE TABLE {destination_schema}.{destionation_table} AS SELECT * FROM df'
    connection_destination.execute(query_write)

# Fermeture des connexions
connection_source.close()
connection_destination.close()

Transfert des tables:   0%|          | 0/19 [00:00<?, ?it/s]

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [ ]:
# Check if the tables are correctly created

In [ ]:
# Check each column for potential casting issues
for col in df.columns:
    try:
        # Create a temporary DataFrame for the single column
        temp_df = df[[col]]
        # Attempt to write the single column to a temp table to identify the problematic column
        temp_table = f'temp_{table}_{col}'
        query_write = f'CREATE OR REPLACE TABLE {destination_schema}_tmp.{temp_table}_{col} AS SELECT * FROM temp_df'
        connection_destination.execute(query_write)
        
    except Exception as e:
        # Log the column causing the issue
        print(f"Column '{col}' in table '{table}' failed with error: {e}")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Column 'vacancy_reasons' in table 'marts_production_housing' failed with error: Invalid Input Error: Failed to cast value: Unimplemented type for cast (VARCHAR -> "NULL")


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [15]:
# run aws cp copy command with os 
import os 

os.system("aws s3 cp ../dbt/metabase.duckdb s3://data-warehouse-raw/metabase.duckdb")

upload failed: ../dbt/metabase.duckdb to s3://data-warehouse-raw/metabase.duckdb An error occurred (InvalidAccessKeyId) when calling the CreateMultipartUpload operation: The AWS Access Key Id you provided does not exist in our records.


256

In [16]:
f"awsc s3 cp ./../dbt/metabase.duckdb  s3://zlv-metabase/")


SyntaxError: unmatched ')' (3349582143.py, line 1)